In [22]:
import numpy as np
import cvxpy as cp

np.set_printoptions(suppress=True, precision=4)

In [23]:
# data
n = 10
m = 45
m_test = 45
sigma = 0.250

train = np.array(
    [
        [1, 2, 1],
        [1, 3, 1],
        [1, 4, 1],
        [1, 5, 1],
        [1, 6, 1],
        [1, 7, 1],
        [1, 8, 1],
        [1, 9, 1],
        [1, 10, 1],
        [2, 3, -1],
        [2, 4, -1],
        [2, 5, -1],
        [2, 6, -1],
        [2, 7, -1],
        [2, 8, -1],
        [2, 9, -1],
        [2, 10, -1],
        [3, 4, 1],
        [3, 5, -1],
        [3, 6, -1],
        [3, 7, 1],
        [3, 8, 1],
        [3, 9, 1],
        [3, 10, 1],
        [4, 5, -1],
        [4, 6, -1],
        [4, 7, 1],
        [4, 8, 1],
        [4, 9, -1],
        [4, 10, -1],
        [5, 6, 1],
        [5, 7, 1],
        [5, 8, 1],
        [5, 9, -1],
        [5, 10, 1],
        [6, 7, 1],
        [6, 8, 1],
        [6, 9, -1],
        [6, 10, -1],
        [7, 8, 1],
        [7, 9, 1],
        [7, 10, -1],
        [8, 9, -1],
        [8, 10, -1],
        [9, 10, 1],
    ],
    dtype=np.int64,
)

test = np.array(
    [
        [1, 2, 1],
        [1, 3, 1],
        [1, 4, 1],
        [1, 5, 1],
        [1, 6, 1],
        [1, 7, 1],
        [1, 8, 1],
        [1, 9, 1],
        [1, 10, 1],
        [2, 3, -1],
        [2, 4, 1],
        [2, 5, -1],
        [2, 6, -1],
        [2, 7, -1],
        [2, 8, 1],
        [2, 9, -1],
        [2, 10, -1],
        [3, 4, 1],
        [3, 5, -1],
        [3, 6, 1],
        [3, 7, 1],
        [3, 8, 1],
        [3, 9, -1],
        [3, 10, 1],
        [4, 5, -1],
        [4, 6, -1],
        [4, 7, -1],
        [4, 8, 1],
        [4, 9, -1],
        [4, 10, -1],
        [5, 6, -1],
        [5, 7, 1],
        [5, 8, 1],
        [5, 9, 1],
        [5, 10, 1],
        [6, 7, 1],
        [6, 8, 1],
        [6, 9, 1],
        [6, 10, 1],
        [7, 8, 1],
        [7, 9, -1],
        [7, 10, 1],
        [8, 9, -1],
        [8, 10, -1],
        [9, 10, 1],
    ],
    dtype=np.int64,
)

# Optional sanity checks:
assert train.shape == (m, 3)
assert test.shape == (m_test, 3)

In [24]:
# transform data
rows = []
for i in range(m):
    row = np.zeros(n)
    t1, t2, won = train[i]
    if won == 1:
        row[t1 - 1] = 1
        row[t2 - 1] = -1
    else:
        row[t1 - 1] = -1
        row[t2 - 1] = 1
    rows.append(row)
A = np.vstack(rows)
A.shape

(45, 10)

In [25]:
a = cp.Variable(n)
objective = cp.Minimize(-cp.sum(cp.log_normcdf(A @ a / sigma)))
constraints = [0 <= a, a <= 1]
problem = cp.Problem(objective, constraints)
problem.solve()
problem.status

'optimal'

In [29]:
a_pred = a.value
a_pred

array([ 1.    , -0.    ,  0.6783,  0.3687,  0.79  ,  0.5813,  0.3874,
        0.0854,  0.67  ,  0.5775])

In [27]:
# prediction on test set
correct = 0
for i in range(m_test):
    t1, t2, won = test[i]
    y_pred = np.sign(a_pred[t1 - 1] - a_pred[t2 - 1])
    if y_pred == won:
        correct += 1
accuracy = correct / m_test
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 86.67%


In [34]:
# skill range increased
b = cp.Variable(n)
objective = cp.Minimize(-cp.sum(cp.log_normcdf(A @ b / sigma)))
constraints = [0 <= b, b <= 4]
problem = cp.Problem(objective, constraints)
problem.solve()
problem.status

'optimal'

In [35]:
b_pred = b.value
b_pred

array([2.9558, 0.0123, 2.0407, 1.6726, 2.1665, 1.9285, 1.7032, 0.8141,
       2.0252, 1.9231])

In [36]:
correct = 0
for i in range(m_test):
    t1, t2, won = test[i]
    y_pred = np.sign(b_pred[t1 - 1] - b_pred[t2 - 1])
    if y_pred == won:
        correct += 1
accuracy = correct / m_test
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 86.67%


In [28]:
# dumb prediction: no change in outcomes
correct_dumb = 0
for i in range(m_test):
    t1, t2, won = test[i]
    y_pred_dumb = train[i][2]
    if y_pred_dumb == won:
        correct_dumb += 1
accuracy_dumb = correct_dumb / m_test
print(f"Dumb Accuracy: {accuracy_dumb * 100:.2f}%")

Dumb Accuracy: 75.56%
